In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
from dataclasses import dataclass, field
from typing import Optional
import huggingface_hub
import functools as ft
import torch
import pandas as pd
import torch
from accelerate import Accelerator
from datasets import load_dataset
from peft import LoraConfig
from tqdm import tqdm
import ast
from transformers import (
    Adafactor,
    AutoTokenizer,
    HfArgumentParser,
    pipeline,
    AutoConfig,
    GPTNeoXForCausalLM,
    AutoModelForCausalLM,
)
from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer, set_seed
from trl.core import LengthSampler
from transformers import pipeline, TextGenerationPipeline
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
from redditqa.dataset import load_reddit_dataset
from dataclasses import dataclass, field
from typing import Optional
import huggingface_hub
import functools as ft
import torch
import pandas as pd
import torch
import os
from accelerate import Accelerator
from datasets import load_dataset
from peft import LoraConfig
from tqdm import tqdm
from transformers import (
    Adafactor,
    AutoTokenizer,
    HfArgumentParser,
    pipeline,
    AutoConfig,
    GPTNeoXForCausalLM,
    AutoModelForCausalLM,
)
from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer, set_seed
from trl.core import LengthSampler
from transformers import (
    pipeline,
    TextGenerationPipeline,
    AutoConfig,
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoModel,
    AutoModelForSequenceClassification,
    GPTNeoXForCausalLM,
    LlamaForSequenceClassification,
)
from redditqa.dataset import load_reddit_dataset
from transformers.utils.hub import convert_file_size_to_int, get_checkpoint_shard_files
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    TaskType,
    get_peft_model,
    set_peft_model_state_dict,
)
from peft.utils import _get_submodules
import peft
import torch
import pandas as pd
from sklearn.metrics import cohen_kappa_score
from statsmodels.stats import inter_rater as irr
import base64
from collections import defaultdict

/workspaces/reddit_qa/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-08-20 13:09:45.625437: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-20 13:09:46.278462: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


#### Loading the eval dataset

In [3]:
eval_dataset = load_reddit_dataset("eval")
eval_dataset = eval_dataset.select(range(200))

Loading cached processed dataset at /scratch1/jhoff/reddit_dataset_cached/eval/cache-cba55e4212677d14.arrow
Loading cached shuffled indices for dataset at /scratch1/jhoff/reddit_dataset_cached/eval/cache-71c7c533e04253a7.arrow


In [4]:
questions = eval_dataset['question_title']

#### Run models

In [5]:
model_checkpoints = {
    # 3B
    '3B_baseline': 'openlm-research/open_llama_3b_v2',
    '3B_cp-2500': '/scratch1/jhoff/checkpoints/finetuned_open-llama-3b-v2/checkpoint-2500_merged',
    # 7B
    '7B_baseline': 'meta-llama/Llama-2-7b-hf',
    '7B_cp-2000': '/scratch1/jhoff/checkpoints/finetuned_llama-2-7b-hf/checkpoint-2000_merged',
    # 7B Chat
    '7B-chat_baseline': 'meta-llama/Llama-2-7b-chat-hf',
    '7B-chat_cp-2500': '/scratch1/jhoff/checkpoints/finetuned_llama-2-7b-chat-hf/checkpoint-2500_merged',
}

Generate answers for questions with the models

In [6]:
template = "<|ELIF|> Question: %question\nAnswer: "

def generate_answers(model_path):

    model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.bfloat16)
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model.config.pad_token_id = model.config.eos_token_id
    model = model.cuda().eval()

    generation_kwargs = {
        "top_k": 0.0,
        "top_p": 1.0,
        "do_sample": True,
        "pad_token_id": tokenizer.pad_token_id,
        "min_length": 32,
        "max_length": 128,
    }

    pipeline = TextGenerationPipeline(model=model, tokenizer=tokenizer, device='cuda:0')

    answers = {}

    for question in tqdm(questions):
        prompt = template.replace('%question', question)
        result = pipeline(prompt, **generation_kwargs, return_full_text=False)
        result = result[0]['generated_text']

        answers[question] = result

    return answers


model_answers = {}
for model_name, checkpoint in model_checkpoints.items():
    model_answers[model_name] = generate_answers(checkpoint)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565, and set the legacy attribute accordingly.
  0%|          | 0/200 [00:00<?, ?it/s]/workspaces/reddit_qa/venv/lib/python3.10/site-packages/transformers/generation/utils.py:1296: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  1%|          | 2/200 [00:03<05:57,  1.81s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end gene

In [7]:
{model_name: list(answers.values())[:3] for model_name, answers in model_answers.items()}

{'3B_baseline': ["50 friends can, by city government's statute. But 50 friends legislating, it's unlikely.\n699 Also:\nLondon has many Chinatowns for your reference. China National Trading (CNTA) or their subsidiaries have also been bidding for real estate in London.\nI have no information about Eric Lyons'",
  '5670 | 417 | 1 | 180 | 1 | 743555 | 695220 | 498032\n11/04/2018\nPosted on 09-04-2018 30-06-2018\n09/04/2018 | 3870 | 2',
  '❤ xxx questions\n'],
 '3B_cp-2500': ['10 cities a year are bordering on tipping point. They are doing things like building free utilities, worker housing, bringing in all types of business to work from home. This will have a really big impact in the future. New Orleans has become a pirate nation. \n\nAlso the city is becoming seen as a social justice issue.',
  "1. they need no resolution. CCTV cameras are primarily used as a deterrent, so a low resolution helps them not be seen during the day. But at night its usually irrelevant because they cant really 

#### Reward scores

Now, let's run our reward model on these generations

In [8]:
# Load the reward model
reward_model_checkpoint = "/scratch1/jhoff/checkpoints/reward_llama-2-7b-chat-hf/checkpoint-3000_merged"

tokenizer = AutoTokenizer.from_pretrained(reward_model_checkpoint)

reward_model = AutoModelForSequenceClassification.from_pretrained(
    reward_model_checkpoint, 
    num_labels=1, 
    torch_dtype=torch.bfloat16
)

reward_pipe = pipeline(
    "sentiment-analysis",
    model=reward_model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device=0,
)

Loading checkpoint shards: 100%|██████████| 3/3 [01:52<00:00, 37.58s/it]


In [9]:
# Apply the reward model to the answers 
template = "<|ELIF|> Question: %question\nAnswer: %answer"

model_rewards = {}
for model_name, model_qas in model_answers.items(): 
    print(f"Applying reward model to answers from {model_name}")
    model_rewards[model_name] = {}
    for question, answer in tqdm(model_qas.items()):
        # Create prompt based on the question and answer
        prompt = template.replace("%question", question).replace("%answer", answer)

        # Apply the reward model to the prompt
        reward = reward_pipe(prompt)[0]["score"]

        # Store the reward
        model_rewards[model_name][question] = reward

Applying reward model to answers from 3B_baseline


100%|██████████| 200/200 [00:10<00:00, 19.57it/s]


Applying reward model to answers from 3B_cp-2500


100%|██████████| 200/200 [00:08<00:00, 24.30it/s]


Applying reward model to answers from 7B_baseline


...

100%|██████████| 200/200 [00:08<00:00, 22.54it/s]


Applying reward model to answers from 7B-chat_baseline


100%|██████████| 200/200 [00:10<00:00, 18.33it/s]


Applying reward model to answers from 7B-chat_cp-2500


100%|██████████| 200/200 [00:09<00:00, 22.05it/s]


Create a dataframe containing the Answers

In [10]:
dfs = {}

for name, qas in model_answers.items():

    rewards = model_rewards[name]

    df = pd.DataFrame.from_records([
        {'question': q, f'{name}.answer': a, f'{name}.reward': rewards[q]} 
        for q, a in qas.items()
    ])

    dfs[name] = df


first_df_name = list(dfs.keys())[0]
df = dfs[first_df_name]
for name, df_other in dfs.items():
    
    if name == first_df_name:
        continue

    df = df.merge(df_other, on='question')

df

,question,3B_baseline.answer,3B_baseline.reward,3B_cp-2500.answer,3B_cp-2500.reward,7B_baseline.answer,7B_baseline.reward,7B_cp-2000.answer,7B_cp-2000.reward,7B-chat_baseline.answer,7B-chat_baseline.reward,7B-chat_cp-2500.answer,7B-chat_cp-2500.reward
0,How does new city formation works in US? If me...,"50 friends can, by city government's statute. ...",0.806853,10 cities a year are bordering on tipping poin...,0.847968,50 people is not enough. 500 people is enough....,0.928409,"\nIf you're in the US, yes. \n\nIt's called [I...",0.948155,|<|ELIF|> Question: How does new city formati...,0.954647,1. You can get a bunch of people to live toget...,0.947381
1,why aren't CCTV cameras made to produce higher...,5670 | 417 | 1 | 180 | 1 | 743555 | 695220 | 4...,0.853913,1. they need no resolution. CCTV cameras are p...,0.913677,1. I don't think that's the case. 2. They're n...,0.819893,\nThe police and businesses do not want to get...,0.868827,There are several reasons why CCTV cameras ar...,0.985936,1. They are cheap\n2. They are not used for le...,0.887205
2,how is Bill Gates still so rich if he is givin...,❤ xxx questions\n,0.791821,\n[LINK]\n\nHe got a lot of money after winnin...,0.853913,1. He is not giving away his money. 2. He is n...,0.948155,1. He still owns a bunch of shares in Microso...,0.927363,Bill Gates is still rich because he has a sig...,0.933431,1. He's been rich for a long time.\n2. He's ma...,0.963780
3,How did America become so powerful despite bei...,1. People really liked independence\n2. Econom...,0.791821,\n\nEventually we will be able to wipe out the...,0.476580,1. They had the most powerful weapon in histor...,0.815233,1) The economic and political power of the Bri...,0.925230,America's rise to power can be attributed to ...,0.975577,"1) When you're young, you're fast.\n\n2) When ...",0.647798
4,How do they make white gold? Can it be melted ...,6D|> |[url=] | | | 0[/url] 0.9\nSigma goes omb...,0.891811,1) Gold is three atoms thick. For the purpose...,0.879974,1. They don’t make white gold. They make gold ...,0.944995,316L is the most common stainless steel used i...,0.954647,White gold is an alloy of gold that is mixed ...,0.968856,1. It's not true gold. White gold is gold with...,0.949669
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,"In a roll of duct tape, how does the glue stic...",<<Rolling the tape smooth..|>\n<<Q.2579|> A.35...,0.858719,"1.) heat, put duct tape into the microwave, fl...",0.795658,"3M makes duct tape, and they will tell you tha...",0.894789,\n[LINK]\n\nI highly recommend watching this a...,0.913677,The glue on a roll of duct tape is applied to...,0.936285,"To me, it's a ""sticky"" adhesive. Not sticky ...",0.853913
196,How do popular social media apps become popula...,1: Storm off with 0 users 2: Use paid advertis...,0.759225,a) They're absurdly good at marketing. Most p...,0.869715,0 users is the point where they have the most ...,0.923039,3rd party apps can use 3rd party APIs to get t...,0.907312,0 users is not necessarily a bad starting poin...,0.975577,1. Start out with the rich and popular kids an...,0.854885
197,Are we fishing at too high of a rate? Are we g...,其个人意见ской。\nSo after doing this quiz and fish\...,0.849971,\n\n I think we have probably caught our catch...,0.865224,100% YES. We have already fished at 100% for t...,0.912436,100 years ago there were so many fish it was s...,0.887205,"Yes, we are fishing at too high of a rate. Th...",0.966410,1) There are *lots* of fish. There are more f...,0.949669
198,What is a freemason and why is it such a secre...,10 out of 10. Great survey about Freemasonry.\...,0.914901,"\nIn idiotic, pop-cultural nonsense, it is jus...",0.662796,1) Freemasonry is the world's largest secret s...,0.893309,13 year old me got a letter from the Freemason...,0.888759,A Freemason is a member of a fraternal organi...,0.973645,"1) Freemasonry is not a secret society, so muc...",0.971129


Show mean reward scores for different checkpoints

In [19]:
reward_cols = [c for c in df.columns if c.endswith('.reward')]
reward_mean = df[reward_cols].mean()
reward_std = df[reward_cols].std()
reward_str = reward_mean.apply(lambda x: f'{x:.4f}').str.cat(reward_std.apply(lambda x: f' ± {x:.4f}'))
reward_str

3B_baseline.reward         0.8310 ± 0.0680
3B_cp-2500.reward          0.8218 ± 0.0907
7B_baseline.reward         0.9049 ± 0.0577
7B_cp-2000.reward          0.9204 ± 0.0534
7B-chat_baseline.reward    0.9645 ± 0.0224
7B-chat_cp-2500.reward     0.9167 ± 0.0551
dtype: object

Review an example

In [20]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):
    display(pd.DataFrame(df.iloc[0]))

,0
question,"How does new city formation works in US? If me and my 50 friend purchase land in middle of nowhere, build homes and start living there, will it be designated as city by state government?"
3B_baseline.answer,"50 friends can, by city government's statute. But 50 friends legislating, it's unlikely.\n699 Also:\nLondon has many Chinatowns for your reference. China National Trading (CNTA) or their subsidiaries have also been bidding for real estate in London.\nI have no information about Eric Lyons'"
3B_baseline.reward,0.806853
3B_cp-2500.answer,"10 cities a year are bordering on tipping point. They are doing things like building free utilities, worker housing, bringing in all types of business to work from home. This will have a really big impact in the future. New Orleans has become a pirate nation. \n\nAlso the city is becoming seen as a social justice issue."
3B_cp-2500.reward,0.847968
7B_baseline.answer,"50 people is not enough. 500 people is enough.\nQuestion: Why is it so?\nAnswer: Because 500 is enough to make a city.\nQuestion: How does new city formation works in US? If me and my 50 friend purchase land in middle of nowhere, build homes and start living there,"
7B_baseline.reward,0.928409
7B_cp-2000.answer,"\nIf you're in the US, yes. \n\nIt's called [Incorporation]([LINK])."
7B_cp-2000.reward,0.948155
7B-chat_baseline.answer,"|<|ELIF|> Question: How does new city formation works in US? If me and my 50 friend purchase land in middle of nowhere, build homes and start living there, will it be designated as city by state government?|<|ELIF|> Answer: New city formation in the US typically involves several steps and"


#### Data for Human Annotators 

Generate sheet to compare models for human annotators. We focus on the baseline and the two checkpoints after 2500 and 8500 steps. We compare to the left and make sure that the order is randomized

In [21]:
def encode_metadata(metadata):
    metadata = repr(metadata)
    metadata = metadata.encode('utf-8')
    metadata = base64.b64encode(metadata)
    return metadata

def decode_metadata(metadata):
    metadata = metadata[2:-1]  # remove b' and '
    metadata = base64.b64decode(metadata)
    metadata = metadata.decode('utf-8')
    metadata = ast.literal_eval(metadata)
    return metadata

In [22]:
def create_comparison(col1, col2): 

    # Create a new dataframe with the two columns
    comparisons = []

    for i, row in df.iterrows():

        question = row['question']
        answer1 = row[col1]
        answer2 = row[col2]

        do_swap = np.random.choice([True, False])


        comparison_row = {
            'question': question,
            'answer1': answer1 if not do_swap else answer2,
            'answer2': answer2 if not do_swap else answer1,
            'metadata': {
                'rand1': np.random.randint(0, 1e5),
                'do_swap': do_swap,
                'answer1': col1 if not do_swap else col2,
                'answer2': col2 if not do_swap else col1, 
                'rand2': np.random.randint(0, 1e5)
            },
        }
        comparison_row['metadata_encoded'] = encode_metadata(comparison_row['metadata'])

        comparisons.append(comparison_row)

    df_comparisons = pd.DataFrame.from_records(comparisons)
    return df_comparisons

#with pd.option_context('display.max_colwidth', None):
#    display(create_comparison('baseline.answer', 'checkpoint-2500.answer'))

In [23]:
df.columns

Index(['question', '3B_baseline.answer', '3B_baseline.reward',
       '3B_cp-2500.answer', '3B_cp-2500.reward', '7B_baseline.answer',
       '7B_baseline.reward', '7B_cp-2000.answer', '7B_cp-2000.reward',
       '7B-chat_baseline.answer', '7B-chat_baseline.reward',
       '7B-chat_cp-2500.answer', '7B-chat_cp-2500.reward'],
      dtype='object')

In [16]:
# Yellow ones
create_comparison('3B_baseline.answer', '3B_cp-2500.answer')\
    .head(100)\
    .to_excel('Model Comparison - 3B Baseline vs 3B SFT (n100).xlsx')

create_comparison('7B_baseline.answer', '7B_cp-2000.answer')\
    .head(100)\
    .to_excel('Model Comparison - 7B Baseline vs 7B SFT (n100).xlsx')

# Green ones
create_comparison('3B_cp-2500.answer', '7B_cp-2000.answer')\
    .head(100)\
    .to_excel('Model Comparison - 3B SFT vs 7B SFT (n100).xlsx')

create_comparison('7B_cp-2000.answer', '7B-chat_cp-2500.answer')\
    .head(100)\
    .to_excel('Model Comparison - 7B SFT vs 7B Chat SFT (n100).xlsx')


# create_comparison('baseline.answer', 'checkpoint-2500.answer')\
#     .to_excel('Model Comparison - Baseline vs Checkpoint-2500 (n200).xlsx')
#
# create_comparison('checkpoint-2500.answer', 'checkpoint-8500.answer')\
#     .to_excel('Model Comparison - Checkpoint-2500 vs Checkpoint-8500 (n200).xlsx')

#### EDA on Data from Human Annotators

First, let's count the votes

In [17]:
comparison_files = [
    'labelled_data/Model Comparison - Baseline vs Checkpoint-2500 (n200).xlsx',
    'labelled_data/Model Comparison - Checkpoint-2500 vs Checkpoint-8500 (n200).xlsx'
]


def organize_answers(row): 
    metadata = row['metadata']
    answer1 = metadata['answer1']
    answer2 = metadata['answer2']

    return {answer1: row['answer1'], answer2: row['answer2']}


answers = {}
for comparison_file in comparison_files:

    # Load comparison data
    comparison_df = pd.read_excel(comparison_file)

    # Decode metadata
    comparison_df["metadata"] = comparison_df.metadata_encoded.apply(decode_metadata)
    comparison_df.drop(columns=["metadata_encoded"], inplace=True)
    comparison_df.metadata = comparison_df.metadata.apply(lambda d: {k: v for k, v in d.items() if k not in ['rand1', 'rand2']})

    # Fix column names
    comparison_df.drop(columns=["assignee1"], inplace=True, errors="ignore")
    comparison_df.rename(columns={"label1": "label"}, inplace=True, errors="ignore")

    # Count votes
    counters = defaultdict(int)
    for _, row in comparison_df.iterrows():
        if row['label'] == '1 - answer1 is better':
            counters[row['metadata']['answer1']] += 1
        
        elif row['label'] == '2 - answer2 is better':
            counters[row['metadata']['answer2']] += 1

        elif row['label'] == '3 - both are equal':
            counters['parity'] += 1

    # Organize answers
    answers_pair = comparison_df.apply(organize_answers, axis=1)
    answers_pair = list(answers_pair)
    model_names = {k for pair in answers_pair for k in pair.keys() }
    for model_name in model_names: 
        answers[model_name] = [pair[model_name] for pair in answers_pair]

    #print(answers_pair[:2])

    print(dict(counters))


# Create df with answers
answers = pd.DataFrame(answers)
#answers.columns = [c.replace('.answer', '') for c in answers.columns]
answers.insert(0, "question", comparison_df["question"])


# Display
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):
    display(answers.head(3))

{'checkpoint-2500.answer': 40, 'baseline.answer': 78, 'parity': 33}
{'checkpoint-2500.answer': 68, 'checkpoint-8500.answer': 60, 'parity': 46}


,question,checkpoint-2500.answer,baseline.answer,checkpoint-8500.answer
0,"How does new city formation works in US? If me and my 50 friend purchase land in middle of nowhere, build homes and start living there, will it be designated as city by state government?","1. There is a very strong rule of thumb that if you can drive to the center of the city in less than 30 minutes, it is not a city.\n\n2. The state will not give you a city charter until you have a population of over 5000 people.\n\n3. People move there.\n\n","|<|ELIF|> Question: How does new city formation works in US? If me and my 50 friend purchase land in middle of nowhere, build homes and start living there, will it be designated as city by state government?\nIn the United States, new city formation typically involves a series of steps and requirements that must be met","1. Is there a population of at least 4000 people living within 4mi of a central point.\n2. Is a permanent settlement of at least 20 people living within 4mi of a central point.\n\nIf you meet both of those criteria, then it is a town.\n\nIf you meet both of those"
1,why aren't CCTV cameras made to produce higher quality footage?,"1) They are not designed to be used as evidence, they are designed to deter crime.\n\n2) They are not affordable.\n\n3) They are not discreet.","🤖 There are several reasons why CCTV cameras are not typically designed to produce higher quality footage. Here are a few:\n1. Cost: Higher quality cameras and recording equipment are more expensive, which can be a significant factor in determining the overall cost of a CCTV system.\n2. Bandwidth: Higher quality video requires more bandwidth to transmit, which can put a strain on network resources and increase the amount of storage","1) Higher quality doesn't necessarily mean better. Sure, 1080p is better than 720p, but 1440p isn't necessarily better than 1080p.\n\n2) Higher quality means more storage space, larger files, and longer recording times. If you don't need any of those things, why pay for them?"
2,how is Bill Gates still so rich if he is giving away most of his money?,1) He still owns quite a bit of Microsoft stock. 2) He has been investing in other companies. 3) He has been giving away a lot of money in the form of grants to charities. 4) He has been involved in some shady business deals (Microsoft antitrust suit) and paid a lot of money to settle those. 5) He has been involved in some shady business deals (Microsoft antit,"Bill Gates is still rich because he has a significant amount of wealth invested in Microsoft stock. Even though he has given away a large portion of his wealth through the Bill and Melinda Gates Foundation, he still owns a significant amount of Microsoft stock, which has continued to increase in value over the years.\n\n\n","He has a lot of money, and he is giving away most of it. He isn't poor. He still has a lot."


Let's recreate the df containing all answers and question from different models

In [18]:
import textstat

for name in answers.columns:

    if name == 'question':
        continue

    text_standard = answers[name].apply(lambda t: textstat.text_standard(t, float_output=True)).mean()
    reading_time = answers[name].apply(textstat.reading_time).mean()

    print(name)
    print(f'  text_standard: {text_standard:.2f}th grade')
    print(f'  reading_time: {reading_time:.2f} seconds')
    print()



ModuleNotFoundError: No module named 'textstat'